In [14]:
import pandas as pd
import numpy as np
import pickle
import random

In [15]:
with open(r'D:\study\thesis\project\HBDM-main\data\datasets\ppi\ppi_index.pkl', 'rb') as f:
    value_to_index_mapping = pickle.load(f)
len(value_to_index_mapping)

18767

In [16]:
local_stringdb = 'D:/study/thesis/project/HBDM-main/data/nn_data/stringdb/'
# load local STRING database and names
df = pd.read_csv(local_stringdb+'9606.protein.info.v12.0.txt', sep='\t', header=0, usecols=['#string_protein_id', 'preferred_name'])
df['preferred_name'] = df['preferred_name'].str.upper()
stringId2name = df.set_index('#string_protein_id')['preferred_name'].to_dict()
name2stringId = df.set_index('preferred_name')['#string_protein_id'].to_dict()
df = pd.read_csv(local_stringdb+'9606.protein.aliases.v12.0.txt', sep='\t', header=0, usecols=['#string_protein_id', 'alias']).drop_duplicates(['alias'], keep='first')
df['alias'] = df['alias'].str.upper()
aliases2stringId = df.set_index('alias')['#string_protein_id'].to_dict()

string_score_transform = lambda x: -np.log(x/1000)

network = pd.read_csv(local_stringdb+'9606.protein.physical.links.detailed.v12.0.txt', sep=' ', header=0).convert_dtypes().replace(0, float('nan'))
network['combined_score'] = network['combined_score'].apply(string_score_transform)

def convert_stringId(alias):
    try:
        stringId = name2stringId[alias]
    except:
        #print(alias, 'can\'t be converted by name2stringId! Now trying aliases2stringId.')
        try:
            stringId = aliases2stringId[alias]
        except:
            #print(alias, 'can\'t be converted by aliases2stringId! Now return None.')
            stringId = None
    #print(alias, stringId)
    return stringId

def read_string_net_from_df(df, weight_choose='combined_score'): # default combined score
    string_net = {}
    for row in df.itertuples(index=False):
        start_node, end_node, weight = row.protein1, row.protein2, getattr(row, weight_choose)
        if start_node not in string_net:
            string_net[start_node] = {}
        string_net[start_node][end_node] = weight
        if end_node not in string_net:
            string_net[end_node] = {}
        string_net[end_node][start_node] = weight
    return string_net

In [17]:
with open(r'D:\study\thesis\project\HBDM-main\data\complexes\complexes.pkl', 'rb') as f:
    complexs = pickle.load(f)
protein_names = list(aliases2stringId.keys())
protein_names.extend(list(name2stringId.keys()))

humans = set(value_to_index_mapping.keys())

complexs_id = dict()

for complex_name in complexs:
    # folder_path = 'D:/study/thesis/project/HBDM-main/ppi_results/test_results/'+complex_name
    # os.mkdir(folder_path)
    group_node = []
    for gene in complexs[complex_name]:
        if gene in protein_names:
            stringid = convert_stringId(gene)
            # stringid = int(stringid[9:])
            # if stringid in humans:
            #     node = value_to_index_mapping[stringid]
            group_node.append(stringid)
    if len(set(group_node)) != 1:
        complexs_id[complex_name] = group_node
    else:
        print(complex_name)


In [18]:
col1 = []
precision_col = []
coverage_col = []
for complex_name in complexs_id:
    col1.append(complex_name)
    points = complexs_id[complex_name]
    for top in [20]:
        precision = []
        coverage = []
        for start_gene in points:
            test_nodes = list(set(points)-set(start_gene))
            true_pre = []
            subdf = network[network['protein1']==start_gene]
            ranked = subdf.sort_values(by='combined_score')
            predicted=ranked[:top]['protein2'].tolist()
            for i in predicted:
                if i in test_nodes:
                    true_pre.append(i)
            precision.append(len(true_pre)/len(predicted))
            coverage.append(len(set(true_pre))/len(test_nodes))
        # print('top-',top,'\t',complex_name,' precision: ', sum(precision)/len(precision))
        # print(complex_name,' coverage: ', sum(coverage)/len(coverage))
        precision_col.append(sum(precision)/len(precision))
        coverage_col.append(sum(coverage)/len(coverage))

In [19]:
len(col1)

173

In [20]:
name = 'sp_detailed_top20'
result = pd.DataFrame({'complex': col1, 'precision_20':precision_col,'coverage_20':coverage_col})
results_name = name +'.csv'
result.to_csv('D:/study/thesis/project/HBDM-main/ppi_results/test_results/complexes/'+results_name,index=False)